# TFM Suministro Global - CREAR RELACIONES: 
##    > CREAR relación Material --[ USADO_EN] -->  Grupo de Aplicación/uso
## _Neo4j - py2neo_
**OBSERVACIONES**  
Ya creada la relación:  Grupos Aplicación/usos --[**INCLUIDO_EN**] -->  Sectores NACE2


## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [5]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [6]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [7]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [8]:
#que es el objeto - tx
# tx = graph_test01.begin()

In [9]:
# tx

In [10]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS previos creados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,91
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [11]:
#validamos la existencia previa de las  relaciones creadas (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal"       
                                  
graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,447


## CARGAR DATOS

In [12]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [14]:
#DATOS: Fichero completo de MATERIAL -> APLICACIONES/USOS -> SECTOR NACE2 APLICACIÓN, dentro del análisis 
# de suministro crítico GLOBAL actual  - Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLAplic_Cargar.csv
#CONTENIDO: Cada Material, su uso y cuota (share) de participación en Grupo de Aplicación/uso, y sector Nace2 vinculado
#           a cada  aplicación/uso   
#USO: Crear las RELACIONES entre "GrupoAplicacion" --[INCLUIDO_EN]--> "SectorAplicacion" y
#     ** Propiedades de [INCLUIDO_EN]: 
#           SHR: proporción decimal de2 dígitos (entre 0 y 1)
#           VAM (importe proporcional de valor añadido - calculado-): SHR por total de Valor Añadido del sector vinculado)   
#     Crear las RELACIONES entre "MaterialREF" --[USADO_EN]--> "GrupoAplicacion" 
#     ** esta relación se crea inicialmente sin propiedades
#
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLAplic_Cargar.csv" 
dfRelAplics = pd.read_csv(rutaFichero, sep=';')


In [15]:
dfRelAplics.head()

,MaterialApl,ApplicationORG,ShareSectorApl,SectorAplMatORG,ValorAnadidoSector,SectorAplicacion,GrupoAplicacion
0,Aluminium,Construction,"0,23",C25 - Manufacture of fabricated metal products...,"148,35",C25-FabrMetalProds-NoMachin-Eq,Building & roads
1,Aluminium,Mobility (Transport and Automotive),"0,21","C29 - Manufacture of motor vehicles, trailers ...","160,60","C29-MotorVehicle,Trailer&SemiT",Transport (automotive)
2,Aluminium,Mobility (Transport and Automotive),"0,21",C30 - Manufacture of other transport equipment,"44,30",C30-other transport equipment,Transport (automotive)
3,Aluminium,Packaging,"0,17",C25 - Manufacture of fabricated metal products...,"148,35",C25-FabrMetalProds-NoMachin-Eq,Packaging
4,Aluminium,High Tech Engineering,"0,12",C28 - Manufacture of machinery and equipment n...,"182,59",C28-Machinery &Equipment n.e.c,High Tech Engineering


In [16]:
dfRelAplics.tail()

,MaterialApl,ApplicationORG,ShareSectorApl,SectorAplMatORG,ValorAnadidoSector,SectorAplicacion,GrupoAplicacion
437,Zirconium,Foundry,"0,15",C24 - Manufacture of basic metals,"55,43",C24-basic metals,Foundry
438,Zirconium,Chemicals,"0,12",C20 - Manufacture of chemicals and chemical pr...,"105,51",C20-Chemicals & Chemc.Prods.,Chemicals
439,Zirconium,Others,"0,10","C26 - Manufacture of computer, electronic and ...","65,70","C26-Computer,Electron.Optic.Ps",Chemicals
440,Zirconium,Pigments,"0,03",C20 - Manufacture of chemicals and chemical pr...,"105,51",C20-Chemicals & Chemc.Prods.,"Chemicals (Paint, Adhesives)"
441,Zirconium,"Superalloys, Nuclear","0,02",C24 - Manufacture of basic metals,"55,43",C24-basic metals,Energy (Nuclear)


In [17]:
dfRelAplics.shape

(442, 7)

In [18]:
dfRelAplics.columns

Index(['MaterialApl', 'ApplicationORG', 'ShareSectorApl', 'SectorAplMatORG',
       'ValorAnadidoSector', 'SectorAplicacion', 'GrupoAplicacion'],
      dtype='object')

### Cargados en memoria las _442 líneas de relación_  entre _material-uso-sector_, para **datos de la cadena de suministro global**

# PREVISUALIZACION DE DATOS - USOS

### Con el fin de procesar de manera más parametrizada y como presentación de otra manera de tratarlo, creo bucle en base a los propios datos de las columnas

In [19]:
strSeparator = "|"
for i in range(0, dfRelAplics.shape[0]):
    strVar = ""
    for j in range(0, dfRelAplics.shape[1]):
        strVar += dfRelAplics[dfRelAplics.columns[j]][i]
        strVar += strSeparator
    print(strVar) 
# se visualiza con el separador específico identificado en la variable  - strSeparator

Aluminium|Construction|0,23|C25 - Manufacture of fabricated metal products, except machinery and equipment|148,35|C25-FabrMetalProds-NoMachin-Eq|Building & roads|
Aluminium|Mobility (Transport and Automotive)|0,21|C29 - Manufacture of motor vehicles, trailers and semi-trailers|160,60|C29-MotorVehicle,Trailer&SemiT|Transport (automotive)|
Aluminium|Mobility (Transport and Automotive)|0,21|C30 - Manufacture of other transport equipment|44,30|C30-other transport equipment |Transport (automotive)|
Aluminium|Packaging|0,17|C25 - Manufacture of fabricated metal products, except machinery and equipment|148,35|C25-FabrMetalProds-NoMachin-Eq|Packaging|
Aluminium|High Tech Engineering|0,12|C28 - Manufacture of machinery and equipment n.e.c.|182,59|C28-Machinery &Equipment n.e.c|High Tech Engineering|
Aluminium|Consumer Durables|0,06|C25 - Manufacture of fabricated metal products, except machinery and equipment|148,35|C25-FabrMetalProds-NoMachin-Eq|Consumer Durables|
Antimony|Flame retardants|0,4

# CREAR RELACION ENTRE  Material Procesado y  Grupo Aplicación
### RELACIÓN: _MaterialRef (MaterialApl) --[USADO_EN]--> GrupoAplicacion_
**Observaciones:**   
(1) Nodos previamente creados      
(2) Relación con DOS propiedades:  
**SHR_MApl** con el valor de ShareSectorApl, o valor entre 0 y 1 con la proporción de aportación de ese material a esa aplicación-sector  
**VAM_MApl** _(CALCULADO)_ = ShareSectorApl x ValorAnadidoSector - Es el valor añadido específico de ese  material para esa aplicación-sector. Por tanto se define como la cantidad proporcional (en millones de euros) que aporta  como valor añadido un material a una aplicación de uso y sector -1 aplicación de uso asignada a 1 sector-)  


### Consulta patrón _CQL-Neo4j_ para crear la relación buscada  
**MATCH**   
    (a:**MaterialREF**),  
    (b:**GrupoAplicacion**)  

**WHERE** a.name = _"materialxxx_REF"_ **AND** b.name = _"grupoaplicacionxxx"_  
**CREATE** (a)**-[r:USADO_EN {SHR_MApl: 0.nn, VAM_MApl: mmmm.mm }]->**(b)  
**RETURN** type(r)

In [20]:
nFilas = dfRelAplics.shape[0]
nFilas 

442

In [21]:
#  Crear RELACION USADO_EN -  Una fila por cada relación, contiene el MaterialREF (MaterialApl) y su GrupoAplicacion  
# (previamente creados ambos nodos) 
#  Nodos: MaterialREF -- para ser asignado a cada aplicación o uso de ese material 
#  Nodos: GrupoAplicacion -- para identificar cada grupo de aplicación de un material procesado/refinado
## PREVISAULIZAR QUERYs antes de la ejecución (visualizar 2 de muestra)
TipoNodoA = "(a:MaterialREF)"
TipoNodoB = "(b:GrupoAplicacion)"
TipoRelacion = " :USADO_EN" 
inicio = 0
nFilas = 2
#
for i in range(inicio, nFilas):
    VAMtmp = round(float(dfRelAplics.ShareSectorApl[i].replace(',', '.')) *               \
                   float(dfRelAplics.ValorAnadidoSector[i].replace(',', '.')), 4)
    strQry = "MATCH "                                        \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfRelAplics.MaterialApl[i]                    \
            + "_REF' AND b.name = '"                       \
            +  dfRelAplics.GrupoAplicacion[i]                                      \
            + "'"                                                                   \
            + " CREATE (a)-[r" + TipoRelacion                                       \
            + " { SHR_MApl: "                                                        \
            + dfRelAplics.ShareSectorApl[i].replace(',', '.')                       \
            + ", VAM_MApl: "                                                        \
            + str( VAMtmp )                                                          \
            + "} ]->(b)"                                                               \
            + " RETURN type(r)"                                 
    print("QRY [" + str(i) + "]> "+ strQry)
# querys  visualizadas
#            + dfRelAplics.ShareSectorApl.replace(',', '.')                                        \
#            + ", VAM_MApl: "                                                           \
#            + dfRelAplics.ValorAnadidoSector.replace(',', '.') * dfRelAplics.ShareSectorApl.replace(',', '.') \


QRY [0]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Building & roads' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.23, VAM_MApl: 34.1205} ]->(b) RETURN type(r)
QRY [1]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Transport (automotive)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.21, VAM_MApl: 33.726} ]->(b) RETURN type(r)


In [22]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')


# Ejecutar 2 relaciones, comprobar y en caso ok, lanzar el resto 

In [23]:
# Crear la RELACIÓN en la BD , una por CADA FILA 
#relación :USADO_EN
## query para crea la relación con sus propiedades 
inicio = 0
nFilas = 2

for i in range(inicio, nFilas):
    VAMtmp = round(float(dfRelAplics.ShareSectorApl[i].replace(',', '.')) *               \
                   float(dfRelAplics.ValorAnadidoSector[i].replace(',', '.')), 4)
    strQry = "MATCH "                                           \
            + TipoNodoA + ", " + TipoNodoB                      \
            + " WHERE a.name = '"                               \
            +  dfRelAplics.MaterialApl[i]                       \
            + "_REF' AND b.name = '"                            \
            +  dfRelAplics.GrupoAplicacion[i]                   \
            + "'"                                               \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_MApl: "                                   \
            + dfRelAplics.ShareSectorApl[i].replace(',', '.')   \
            + ", VAM_MApl: "                                    \
            + str( VAMtmp )                                     \
            + "} ]->(b)"                                        \
            + " RETURN type(r)"                                 
#
# ejecutar Query en BD abierta - Neo4j
#
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [0]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Building & roads' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.23, VAM_MApl: 34.1205} ]->(b) RETURN type(r)
EJECUTADA QRY [1]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Transport (automotive)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.21, VAM_MApl: 33.726} ]->(b) RETURN type(r)


In [24]:
# guardar como dataframe el resultado de buscar todas las relaciones creadas de tipo USADO_EN
# de momento sólo están creadas las 2 primeras, y etiquetamos el resultado con el nombre de la relación 
dfGraphMat= graph_test01.run("MATCH p=()-[r:USADO_EN]->() RETURN (p) as USADO_EN").to_data_frame()

In [25]:
# visualizar el dataframe generado con .to_data_frame() perteneciente a la funcionalidad de la librería py2neo
dfGraphMat


,USADO_EN
0,"({'VAM_MApl': 34.1205, 'SHR_MApl': 0.23})"
1,"({'VAM_MApl': 33.726, 'SHR_MApl': 0.21})"


In [26]:
dfRelAplics.shape

(442, 7)

### Creamos el resto de las relaciones para las restantes 445 filas (rango desde 2 a 447 ) 
una vez que se verifica la creación de las 2 primeras filas

In [27]:
# Crear la RELACIÓN en la BD , una por CADA FILA (****restantes ***) 
#relación USADO_EN
## query para crea la relación con sus propiedades (en este caso no aplica)
# VAMtmp contiene para cada fila el valor calculado para la propiedad VAM_MApl
#
inicio = nFilas
nFilas = dfRelAplics.shape[0]
 
for i in range(inicio, nFilas):
    VAMtmp = round(float(dfRelAplics.ShareSectorApl[i].replace(',', '.')) *               \
                   float(dfRelAplics.ValorAnadidoSector[i].replace(',', '.')), 4)
    strQry = "MATCH "                                           \
            + TipoNodoA + ", " + TipoNodoB                      \
            + " WHERE a.name = '"                               \
            +  dfRelAplics.MaterialApl[i]                       \
            + "_REF' AND b.name = '"                            \
            +  dfRelAplics.GrupoAplicacion[i]                   \
            + "'"                                               \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_MApl: "                                   \
            + dfRelAplics.ShareSectorApl[i].replace(',', '.')   \
            + ", VAM_MApl: "                                    \
            + str( VAMtmp )                                     \
            + "} ]->(b)"                                        \
            + " RETURN type(r)"                                 
#
# ejecutar Query en BD abierta - Neo4j
#
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [2]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Transport (automotive)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.21, VAM_MApl: 9.303} ]->(b) RETURN type(r)
EJECUTADA QRY [3]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Packaging' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.17, VAM_MApl: 25.2195} ]->(b) RETURN type(r)
EJECUTADA QRY [4]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'High Tech Engineering' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.12, VAM_MApl: 21.9108} ]->(b) RETURN type(r)
EJECUTADA QRY [5]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Aluminium_REF' AND b.name = 'Consumer Durables' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.06, VAM_MApl: 8.901} ]->(b) RETURN type(r)
EJECUTADA QRY [6]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Antimony_REF' AND b.name = 'Chemicals' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.43, VAM_MApl

EJECUTADA QRY [48]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Borate_REF' AND b.name = 'Building & roads' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.04, VAM_MApl: 4.2204} ]->(b) RETURN type(r)
EJECUTADA QRY [49]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Borate_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.04, VAM_MApl: 2.2172} ]->(b) RETURN type(r)
EJECUTADA QRY [50]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Borate_REF' AND b.name = 'Others' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.00, VAM_MApl: 0.0} ]->(b) RETURN type(r)
EJECUTADA QRY [51]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Cadmium_REF' AND b.name = 'Batteries' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.80, VAM_MApl: 64.6} ]->(b) RETURN type(r)
EJECUTADA QRY [52]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Cadmium_REF' AND b.name = 'Chemicals (Paint, Adhesives)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.11, VAM_MApl: 11.6061} ]->(b) 

EJECUTADA QRY [101]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Feldspar_REF' AND b.name = 'Ceramics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.45, VAM_MApl: 25.767} ]->(b) RETURN type(r)
EJECUTADA QRY [102]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Feldspar_REF' AND b.name = 'Glass' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.06, VAM_MApl: 3.4356} ]->(b) RETURN type(r)
EJECUTADA QRY [103]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Feldspar_REF' AND b.name = 'Building & roads' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.46, VAM_MApl: 26.3396} ]->(b) RETURN type(r)
EJECUTADA QRY [104]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Feldspar_REF' AND b.name = 'Chemicals' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.03, VAM_MApl: 0.0} ]->(b) RETURN type(r)
EJECUTADA QRY [105]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Fluorspar_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.36, VAM_MApl: 19.9548} ]->(b) RETU

EJECUTADA QRY [149]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Indium_REF' AND b.name = 'Electrical and Electronics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.60, VAM_MApl: 39.42} ]->(b) RETURN type(r)
EJECUTADA QRY [150]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Indium_REF' AND b.name = 'Electrical and Electronics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.11, VAM_MApl: 7.227} ]->(b) RETURN type(r)
EJECUTADA QRY [151]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Indium_REF' AND b.name = 'Energy (Solar)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.09, VAM_MApl: 5.913} ]->(b) RETURN type(r)
EJECUTADA QRY [152]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Indium_REF' AND b.name = 'Thermal interface material' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.07, VAM_MApl: 4.599} ]->(b) RETURN type(r)
EJECUTADA QRY [153]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Indium_REF' AND b.name = 'Batteries' CREATE (a)-[r :USADO_EN { SHR_MApl:

EJECUTADA QRY [198]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Lithium_REF' AND b.name = 'Glass' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.66, VAM_MApl: 37.7916} ]->(b) RETURN type(r)
EJECUTADA QRY [199]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Lithium_REF' AND b.name = 'Lubricants' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.09, VAM_MApl: 9.4959} ]->(b) RETURN type(r)
EJECUTADA QRY [200]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Lithium_REF' AND b.name = 'Building & roads' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.09, VAM_MApl: 5.1534} ]->(b) RETURN type(r)
EJECUTADA QRY [201]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Lithium_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.05, VAM_MApl: 2.7715} ]->(b) RETURN type(r)
EJECUTADA QRY [202]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Lithium_REF' AND b.name = 'Pharmacy' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.04, VAM_MApl: 3.2072} ]->(b) RETURN 

EJECUTADA QRY [252]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Natural Rubber_REF' AND b.name = 'Transport (automotive)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.75, VAM_MApl: 120.45} ]->(b) RETURN type(r)
EJECUTADA QRY [253]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Natural Rubber_REF' AND b.name = 'Transport (automotive)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.14, VAM_MApl: 6.202} ]->(b) RETURN type(r)
EJECUTADA QRY [254]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Natural Rubber_REF' AND b.name = 'Machinery' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.06, VAM_MApl: 10.9554} ]->(b) RETURN type(r)
EJECUTADA QRY [255]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Natural Rubber_REF' AND b.name = 'Furniture' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.02, VAM_MApl: 0.5234} ]->(b) RETURN type(r)
EJECUTADA QRY [256]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Natural Rubber_REF' AND b.name = 'Packaging' CREATE (a)-[r :USADO_

EJECUTADA QRY [296]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Phosphorus_REF' AND b.name = 'Agriculture' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.04, VAM_MApl: 4.2204} ]->(b) RETURN type(r)
EJECUTADA QRY [297]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Platinum_REF' AND b.name = 'Catalysts' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.77, VAM_MApl: 123.662} ]->(b) RETURN type(r)
EJECUTADA QRY [298]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Platinum_REF' AND b.name = 'Jewelery & others' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.10, VAM_MApl: 3.916} ]->(b) RETURN type(r)
EJECUTADA QRY [299]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Platinum_REF' AND b.name = 'Chemicals' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.06, VAM_MApl: 6.3306} ]->(b) RETURN type(r)
EJECUTADA QRY [300]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Platinum_REF' AND b.name = 'Others' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.00, VAM_MApl: 0.0} ]->(b) RETU

EJECUTADA QRY [343]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Silica sand_REF' AND b.name = 'Glass' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.31, VAM_MApl: 17.7506} ]->(b) RETURN type(r)
EJECUTADA QRY [344]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Silica sand_REF' AND b.name = 'Chemicals' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.16, VAM_MApl: 9.1616} ]->(b) RETURN type(r)
EJECUTADA QRY [345]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Silica sand_REF' AND b.name = 'Foundry' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.13, VAM_MApl: 7.2059} ]->(b) RETURN type(r)
EJECUTADA QRY [346]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Silica sand_REF' AND b.name = 'Filler, extender, sealant' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.03, VAM_MApl: 2.2794} ]->(b) RETURN type(r)
EJECUTADA QRY [347]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Silica sand_REF' AND b.name = 'Energy (Oil-Gas)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.00, VA

EJECUTADA QRY [387]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Tantalum_REF' AND b.name = 'Hard materials (tools)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.10, VAM_MApl: 18.259} ]->(b) RETURN type(r)
EJECUTADA QRY [388]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Tantalum_REF' AND b.name = 'Industry' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.04, VAM_MApl: 5.934} ]->(b) RETURN type(r)
EJECUTADA QRY [389]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Tellurium_REF' AND b.name = 'Energy (Solar)' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.40, VAM_MApl: 26.28} ]->(b) RETURN type(r)
EJECUTADA QRY [390]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Tellurium_REF' AND b.name = 'Thermo-electric devices' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.30, VAM_MApl: 19.71} ]->(b) RETURN type(r)
EJECUTADA QRY [391]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Tellurium_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.15, 

EJECUTADA QRY [430]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Zinc_REF' AND b.name = 'Galvanising' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.52, VAM_MApl: 77.142} ]->(b) RETURN type(r)
EJECUTADA QRY [431]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Zinc_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.17, VAM_MApl: 9.4231} ]->(b) RETURN type(r)
EJECUTADA QRY [432]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Zinc_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.15, VAM_MApl: 8.3145} ]->(b) RETURN type(r)
EJECUTADA QRY [433]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Zinc_REF' AND b.name = 'Chemicals' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.10, VAM_MApl: 10.551} ]->(b) RETURN type(r)
EJECUTADA QRY [434]> MATCH (a:MaterialREF), (b:GrupoAplicacion) WHERE a.name = 'Zinc_REF' AND b.name = 'Metals-Basics' CREATE (a)-[r :USADO_EN { SHR_MApl: 0.06, VAM_MApl: 8.901} ]->(b) RETURN type(r)
EJ

In [28]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[4] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal"  
                                  
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,91
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [30]:
#validamos que todas las RELACIONES se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal    \
UNION MATCH m=()-[p:USADO_EN]->()                      \
RETURN type(p) as TipoRelacion, count (p) as Ntotal"       
                                  
graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442
1,USADO_EN,442


# fin proceso - relación  "INCLUIDO_EN" - CREADA

In [81]:
# fin archivo